In [1]:
from utils.df_handle import *

Default bq project:  spatial-vision-343005.biteam


In [2]:
df=pd.read_csv("D:/software/ecommerce/backend/base/testdata.csv")

In [5]:
import json

In [6]:
df.to_json(orient="records")

'[{"name":"Duy","soluong":10,"brand":"d"},{"name":"Huy","soluong":20,"brand":"h"}]'

In [8]:
parsed = json.loads(df.to_json(orient="records"))

In [14]:
df.to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>name</th>\n      <th>soluong</th>\n      <th>brand</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td>Duy</td>\n      <td>10</td>\n      <td>d</td>\n    </tr>\n    <tr>\n      <th>1</th>\n      <td>Huy</td>\n      <td>20</td>\n      <td>h</td>\n    </tr>\n  </tbody>\n</table>'

In [3]:
deli = pd.read_csv('DELI.csv', usecols=['BranchID', 'OrderNbr', 'SlsperID'])

In [4]:
deli.columns

Index(['BranchID', 'SlsperID', 'OrderNbr'], dtype='object')

In [2]:
datenow = datetime.now().strftime("%Y%m%d")
param_2 = f"'20161130'"
param_3 = f"'{datenow}'"

In [3]:
if datetime.now().hour in {8,9,10,11,12,13,14,15,16,17,18,22}:
    print ("OK")

OK


In [4]:
query = f"EXEC [pr_AR_TrackingDebtConfirm] @Fromdate={param_2}, @Todate={param_3}"
start=time.time()
df1 = get_ms_df(sql=query, parse_dates={"DateOfOrder": {"dayfirst": True}, "DueDate": {"dayfirst": True}, "OrderDate": {"dayfirst": True}})
# df1.to_csv('doanhthutienmat_data1.csv', index = False)
end=time.time()
duration=end-start
print(duration)

47.875386238098145


In [ ]:
datenow = datetime.now().strftime("%Y%m%d")
print(datenow)

In [10]:
df_nhansu = get_ps_df("select * from d_nhan_su")
df_nhansu['qlkhuvuc'] = df_nhansu['qlkhuvuc'].str.strip()
df_nhansu_asm = dropdup(df_nhansu[['manvcrscrss','qlkhuvuc']], 1)
nhansu_asm_dict = df_to_dict(df_nhansu_asm)
df_phu_trach_no_cs_theo_tinh = get_ps_df("SELECT * from d_phu_trach_no_cs_theo_tinh")
phu_trach_no_cs_theo_tinh_dict = df_to_dict(df_phu_trach_no_cs_theo_tinh)
df1['ASMName'] = np.where(df1['DebtInCharge']=='MDS', df1['SlsperID'].map(nhansu_asm_dict).fillna('Lương Trịnh Thắng (KN)'), df1['State'].map(phu_trach_no_cs_theo_tinh_dict))

In [11]:
# ((df1['RefCustId'].isin(list_cs)) & (df1['DebtInCharge']=='CS'))

In [12]:
ma_kh_cu_dict = get_ms_df("SELECT CustId, RefCustID from AR_Customer").set_index('CustId').to_dict()['RefCustID']
df1['RefCustId'] = df1['CustID'].map(ma_kh_cu_dict)
df_phu_trach_no_cs_theo_refcustid = get_ps_df("select * from d_phu_trach_no_cs_theo_refcustid")
phu_trach_no_cs_theo_refcustid_dict = get_ps_df("select * from d_phu_trach_no_cs_theo_refcustid").set_index('refcustid').to_dict()['inchargename']
list_cs = df_phu_trach_no_cs_theo_refcustid['refcustid'].to_list()
df1['ASMName_CS'] = df1['RefCustId'].map(phu_trach_no_cs_theo_refcustid_dict)
df1['ASMName'] = np.where( ((df1['RefCustId'].isin(list_cs)) & (df1['DebtInCharge']=='CS')), df1['ASMName_CS'], df1['ASMName'])

In [13]:
df_nhansu_sup = dropdup(df_nhansu[['manvcrscrss','quanlytructiep']], 1)
df_nhansu_sup['quanlytructiep'] = df_nhansu_sup['quanlytructiep'].str.strip()
df1['SupName'] = df1['SupName'].map( df_to_dict(df_nhansu_sup) ).fillna(df1['ASMName'])
df_nhansu_rsm = dropdup(df_nhansu[['manvcrscrss','qlvung']], 1)
df_nhansu_rsm['qlvung'] = df_nhansu_rsm['qlvung'].str.strip()
df1['SupName'] = df1['SupName'].map( df_to_dict(df_nhansu_sup) ).fillna(df1['ASMName'])
df1['RSMName'] = df1['RSMName'].map( df_to_dict(df_nhansu_rsm) ).fillna(df1['ASMName'])
df1['InChargeName'] =  np.where(df1['DebtInCharge']=='MDS', df1['SlsperName'], df1['ASMName'])
df_vptt = get_ps_df("select * from d_vptt")
df1 = df1.merge(df_vptt, how = 'left', left_on='State', right_on='tinh', suffixes=('_left', '_right'), validate="m:1")
df_mkv_viet_tat = get_ps_df("select * from d_mkv_viet_tat").set_index('tenkhuvuc')
khuvuc_dict = df_mkv_viet_tat.to_dict()['khuvuc']
df1['Territory'] = df1['Territory'].map(khuvuc_dict)
df1['Position'] = np.where(df1['DebtInCharge']=="CS", "CS", df1['Position'])
df1[checkdup(df1, 2, ['BranchID','OrderDate','OrderNbr','DateOfOrder','DueDate'])]

In [34]:
groupbylist = [
    'OrderNbr', 
    'BranchID', 
    'Position',
    'SlsperID', 
    'SupName', 
    'ASMName', 
    'RSMName', 
    'DateOfOrder', 
    'DueDate',
    'CustID',
    'RefCustId',
    'CustName',
    'SlsperName',
    'InChargeName',
    'DebtInCharge',
    'Terms', 
    'PaymentsForm',
    'TermsType',
    'Territory', 
    'State', 
    'vptt', 
    'DeliveryUnit',
    'Channels',
    'SubChannel'
    ]

In [35]:
aggregate_dict = {
'OrderDate': np.max,
#Group by tien
'OpeningOrderAmt':np.sum,
'OrdAmtRelease':np.sum,
'DeliveredOrderAmt':np.sum,
'ReturnOrdAmt': np.sum,
'DebtConfirmAmt': np.sum,
'DebtConfirmAmtRelease': np.sum,
# Huy & Xac Nhan In Month
# 'ReturnOrdAmt_InMonth': np.sum,
# 'DebConfirmAmtRelease_InMonth': np.sum,

#DonHang8
'CountOpeningOrder':np.sum,
'CountOrdRelease': np.sum,
'DeliveredOrder':np.sum,
'CountReturnOrd': np.sum,
'CountDebtConfirm': np.sum,
'CountDebtConfirmRelease':np.sum
}

In [51]:
df2 = pivot(df1, groupbylist, aggregate_dict)

In [55]:
rename_dict ={
# Doanh So
'OpeningOrderAmt': 'tiennodauky',
'OrdAmtRelease':'tienchotso',
'DeliveredOrderAmt':'tiengiaothanhcong',
'ReturnOrdAmt':'tienhuydon',
'DebtConfirmAmt':'tienlenbangke',
'DebtConfirmAmtRelease':'tienthuquyxacnhan',
# Huy & Xac Nhan In Month
# 'ReturnOrdAmt_InMonth': 'tienhuydon_inmonth',
# 'DebConfirmAmtRelease_InMonth': 'tienthuquyxacnhan_inmonth',
# Don Hang
'CountOpeningOrder': 'dondauky',
'CountOrdRelease':'donchotso',
'DeliveredOrder':'dongiaothanhcong',
'CountReturnOrd':'donhuy',
'CountDebtConfirm':'donlenbangke',
'CountDebtConfirmRelease':'donthuquyxacnhan'
}
df2.rename(columns=rename_dict, inplace=True)
ctr1 = df2['DeliveryUnit']=='Nhà vận chuyển'
ctr2 = df2['donhuy']==0
df2['dongiaothanhcong'] = np.where(ctr1&ctr2, 1, df2['dongiaothanhcong'])
ctr1 = df2['DeliveryUnit']=='Nhà vận chuyển'
ctr2 = df2['tienhuydon']==0
df2['tiengiaothanhcong'] = np.where(ctr1&ctr2, df2['tienchotso'], df2['tiengiaothanhcong'])
df2['tiennocongty'] = df2['tiennodauky'] + df2['tienchotso'].values - df2['tienhuydon'].values - df2['tienthuquyxacnhan'].values
df2['donnocongty'] = np.where(df2['tiennocongty'].values > 0, 1, 0)
df2['donchuagiao'] = df2['donchotso'] - df2['dongiaothanhcong'] - df2['donhuy']
df2['tiendonchuagiao'] = df2['tienchotso'] - df2['tiengiaothanhcong'] - df2['tienhuydon']

In [44]:
# df2.columns

Index(['ordernbr', 'branchid', 'position', 'slsperid', 'supname', 'asmname',
       'rsmname', 'dateoforder', 'duedate', 'custid', 'refcustid', 'custname',
       'slspername', 'inchargename', 'debtincharge', 'terms', 'paymentsform',
       'termstype', 'territory', 'state', 'vptt', 'deliveryunit', 'channels',
       'subchannel', 'orderdate', 'openingorderamt', 'ordamtrelease',
       'deliveredorderamt', 'returnordamt', 'debtconfirmamt',
       'debtconfirmamtrelease', 'countopeningorder', 'countordrelease',
       'deliveredorder', 'countreturnord', 'countdebtconfirm',
       'countdebtconfirmrelease'],
      dtype='object')

In [45]:
# df3.tiendonchuagiao.sum()

In [46]:
# df2.columns = [x.lower() for x in df2.columns]

In [47]:
# df2[checkdup(df2, 2, ['ordernbr', 'branchid', 'dateoforder', 'duedate'])].to_clipboard()

In [48]:
# df2.columns = [x.lower() for x in df2.columns]
trangthaigiaohangdict = {'C':'Đã giao hàng', 'H':'Chưa Xác Nhận', 'D':'KH Không Nhận', 'A':'Đã Xác Nhận', 'R':'Từ Chối Giao Hàng', 'E':'Không Tiếp Tục Giao Hàng'}
# get_ms_csv(DELI, 'DELI.csv')
DELI=pd.read_csv('DELI.csv', usecols=['BranchID', 'OrderNbr', 'Status'])
DELI['BranchID_OrderNbr'] = DELI['BranchID']+DELI['OrderNbr']
# DELI.columns
DELI = DELI[['BranchID_OrderNbr','Status']]
# df2.columns
df2['trangthaigiaohang'] = (df2['branchid']+df2['ordernbr']).map(df_to_dict(DELI)).map(trangthaigiaohangdict)
# vc(df2,'deliveryunit')

# update 21/02
dk1 = df2.DeliveryUnit.isna()
dk2 = df2.DateOfOrder < datetime(2021,5,1)
df2.trangthaigiaohang = np.where( dk1 & dk2, 'Đã giao hàng', df2.trangthaigiaohang)
df2.deli_last_updated = np.where( dk1 & dk2, df2.DateOfOrder, df2.deli_last_updated)

dk1 = df2.DeliveryUnit == 'Nhà vận chuyển'
df2.trangthaigiaohang = np.where( dk1, 'Giao NVC', df2.trangthaigiaohang)
df2.deli_last_updated = np.where( dk1, df2.DateOfOrder, df2.deli_last_updated)

# 10/02/2022 UPDATE
DELI=pd.read_csv(csv_path+'DELI.csv', usecols=['BranchID', 'OrderNbr', 'SlsperID'])
DELI.columns = ['BranchID','OrderNbr','manvgh']
df2.merge(DELI, on = ['BranchID','OrderNbr'], how = 'left')

In [50]:
# df2=df2[['ordernbr', 'branchid', 'dateoforder', 'duedate', 'tienhuydon_inmonth','tienthuquyxacnhan_inmonth']]

In [51]:
primary_keys=['ordernbr', 'branchid', 'dateoforder', 'duedate']

In [52]:
df2['inserted_at'] = datetime.now()

In [54]:
dropdup(df2,1,['ordernbr', 'branchid', 'dateoforder', 'duedate'])

,ordernbr,branchid,position,slsperid,supname,asmname,rsmname,dateoforder,duedate,custid,...,tienlenbangke,tienthuquyxacnhan,dondauky,donchotso,dongiaothanhcong,donhuy,donlenbangke,donthuquyxacnhan,trangthaigiaohang,inserted_at
0,DH012017-12644,MR0003,CS,MR2168,Lương Tấn Khả,Lương Tấn Khả,Lương Tấn Khả,2017-01-18,2017-04-18,TD42I014,...,0.0,0.0,1,0,0,0,0,0,NaN,2021-11-11 15:28:41.303347
1,DH012017-13345,MR0001,CS,MR0456,Lương Tấn Khả,Lương Tấn Khả,Lương Tấn Khả,2017-01-18,2017-04-18,N02209270,...,0.0,0.0,1,0,0,0,0,0,NaN,2021-11-11 15:28:41.303347
2,DH012017-13346,MR0001,CS,MR0456,Lương Tấn Khả,Lương Tấn Khả,Lương Tấn Khả,2017-01-18,2017-04-18,N02209270,...,0.0,0.0,1,0,0,0,0,0,NaN,2021-11-11 15:28:41.303347
3,DH012017-13360,MR0001,CS,MR0456,Lương Tấn Khả,Lương Tấn Khả,Lương Tấn Khả,2017-01-18,2017-04-18,N02209270,...,0.0,0.0,1,0,0,0,0,0,NaN,2021-11-11 15:28:41.303347
4,DH012019-05622,MR0003,CS,MR2168,Lê Thị Mai Phương,Lê Thị Mai Phương,Lê Thị Mai Phương,2019-01-10,2019-04-10,TD30I028,...,0.0,0.0,1,0,0,0,0,0,NaN,2021-11-11 15:28:41.303347
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106596,DH122020-39619,MR0013,CS,MR1768,Đinh Thị Ngọc Mẫn,Đinh Thị Ngọc Mẫn,Đinh Thị Ngọc Mẫn,2020-12-30,2021-03-30,N0110332,...,7400000.0,7400000.0,1,0,0,0,1,1,NaN,2021-11-11 15:28:41.303347
106597,DH122020-39846,MR0010,CS,MR2000,Lê Thị Mai Phương,Lê Thị Mai Phương,Lê Thị Mai Phương,2020-12-30,2021-03-30,HH04E001,...,3700001.0,3700001.0,1,0,0,0,1,1,NaN,2021-11-11 15:28:41.303347
106598,DH122020-39876,MR0001,CS,MR2081,Lương Tấn Khả,Lương Tấn Khả,Lương Tấn Khả,2020-12-30,2021-03-30,TN90I018,...,0.0,0.0,1,0,0,0,0,0,NaN,2021-11-11 15:28:41.303347
106599,DH122020-40762,MR0001,CS,MR0246,Phạm Thị Cẩm Tú,Phạm Thị Cẩm Tú,Phạm Thị Cẩm Tú,2020-12-31,2021-03-31,M100902,...,1920000.0,1920000.0,1,0,0,0,1,1,NaN,2021-11-11 15:28:41.303347


In [56]:
df2[checkdup(df2, 2, ['ordernbr', 'branchid', 'dateoforder', 'duedate'])]

,ordernbr,branchid,position,slsperid,supname,asmname,rsmname,dateoforder,duedate,custid,...,tienlenbangke,tienthuquyxacnhan,dondauky,donchotso,dongiaothanhcong,donhuy,donlenbangke,donthuquyxacnhan,trangthaigiaohang,inserted_at
105655,DH112021-01987,MR0001,CS,MR2081,Lê Thị Mai Phương,Lê Thị Mai Phương,Lê Thị Mai Phương,2021-11-10,2022-02-08,TN90I048,...,0.0,0.0,0,1,0,0,0,0,NaN,2021-11-11 15:28:41.303347
105656,DH112021-01987,MR0001,CS,MR2081,Lê Thị Mai Phương,Lê Thị Mai Phương,Lê Thị Mai Phương,2021-11-10,2022-02-08,TN90I048,...,0.0,0.0,0,0,1,0,0,0,NaN,2021-11-11 15:28:41.303347


In [ ]:
commit_psql("truncate table f_tracking_debt cascade;")

In [ ]:
execute_values_upsert(df2, 'f_tracking_debt', pk=primary_keys)